Ce notebook contient le code de l'entrainement des deux modèles. Il vient en complément du notebook.ipynb dans le même dépôt

## 4. Entrainement des 2 modèles de générations données
---
`Fine tune GPT model for poeme generator on the dataset`

Dans cette partie, nous allons entrainer 2 modèles pour la génération de poème. Un premier modèle GPT-2 qui utilise l'architecture de transformer (l'état de l'art dans le NLP) mais qui demande d'avoir assez de ressource pour arriver à des bons résultats. Ce modèle ne sera pas entrainé from scratch, on va fine tune un modèle pré-entrainer récupéré sur hugging face. Grâce à l'API qu'offre hugging face pour fine tuning les Transformers, en l'occurrence ici GPT-2, celui-ci nous a rendu la tâche simple pour fine tune notre modèle.

Le deuxième modèle que nous allons entrainer sera un RNN, qui est un modèle auto-régressif. Avec les deux modèles, on fera une analyse sur les résultats que nous obtiendrons


- !!!
---
Nous aurions pu utiliser le dernier modèle GPT-3, mais ce dernier reste utilisable via une api en ligne, on a donc fait le choix d'utiliser le gpt-2. Fine-tune GPT-3 pour cette tâche pourrait être une piste d'amélioration.

In [1]:
!pip install transformers

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel
from transformers.utils.dummy_pt_objects import BeamScorer
import gc

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device 

'cuda'

In [4]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

0

---
Plutôt que d'utiliser un modèle gpt-2 entrainer par les équipes de Googles, qui est entrainé sur les données en anglais, nous avons fait le choix de fine-tune un modèle entrainer sur les données en Français

---

In [5]:
# Gpt-2 english model
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', bos_token='<|startoftext|>',
#                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
#model = GPT2LMHeadModel.from_pretrained('gpt2-medium').to(device)
#model.resize_token_embeddings(len(tokenizer))

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained("antoiloui/belgpt2")
model = GPT2LMHeadModel.from_pretrained("antoiloui/belgpt2").to(device)
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/951k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/519k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Embedding(50257, 768)

- Fonction pour la génération d'un texte (poème)

In [12]:
def generate_text(text, num_return_sequences = 1):
    
    generated = tokenizer(text, return_tensors="pt").input_ids.to(device)
    
    sample_outputs = model.generate(generated, 
                                    do_sample=True, 
                                    top_k=50, 
                                    max_length=300, 
                                    top_p=0.95, 
                                    temperature=1.9, 
                                    num_return_sequences=num_return_sequences)
    
    for i, sample_output in enumerate(sample_outputs):
        print('Text ',i)
        print('------------------------------------------')
        print("{}".format(tokenizer.decode(sample_output, skip_special_tokens=True)))

In [13]:
p1 = "L'amour est un oiseau rebel"
p2 = "Science sans conscience n'est que ruine de l'ame "
p3 = "La haine que l'on ressent "

generate_text(p1,2)

In [14]:
generate_text(p1,2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text  0
------------------------------------------
L'amour est un oiseau rebel ; une seule parole peut m' élever jusqu' à Dieu de cette vie solitaire mais qui, après l' avoir donnée aux vivants avec joie, s' y réfugie sous une douce tyrannie, de même jusqu' à sa mort. Je te demande seulement ce qui te chagrine parce quel que vous soyez ce qui t'étonne pas de la politique. LDLC pour les petits epub de toute catégorie : à vous de venir découvrir la plateforme du leader et ceux déjà en activité ainsi que sa politique commerciale avec les nouveaux lecteurs qui vont se rajouter prochainement. Dans mon souvenir d' amateur de cinéma, c' étaient en fait la première trilogie du réalisateur suédois Carl Ove Andersen, ainsi que la 2. Avec les températures au thermomètre qui descendent jusqu' aux 4, 5 pour 10 degrés, cela fait beaucoup : la nourriture qui mijote devient une denrée de première nécessité. En raison des incertitudes liées à notre analyse des prix, aucune confirmation de commercialisa

In [15]:
generate_text(p2,2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text  0
------------------------------------------
Science sans conscience n'est que ruine de l'ame ʾHan hījūs, dont aucun autre pays nen jouit a fait fortune. Une fois qu' on y a pensé une deuxième couche vous les déplacez. Je vais faire ce chemin qui permet à mon cheval de revenir au moins en forme le 1 juin prochain lors des essais de dimanche prochain avec un système à puce et des réglages dans lequel je pourrai continuer pendant cinq mois et je verrai tout le temps que j' aimerai retrouver en course au prochain grand tour final en 2015 pour préparer de belle épreuve.Pensez donc à l avenir, nous pensons au championnat d ouverture qu est le GP d Autriche mais je crois qu à cette heure la dernière épreuve pour moi sera vraiment d ici la moitié début Novembre afin de vous motiver si le besoin me tenaille d ores et déjà avant même les 3e et 4e manche mais le week-end précédant mon Tour de Suisse vous y serez bien souvent déjà.Via mes blogs photos des vacances en Argentine, ce sont des 

In [16]:
generate_text(p3,2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text  0
------------------------------------------
La haine que l'on ressent ourdît au lieu que la liberté de presse vous donnast un libre appui à votre cause. Le film est basé à Melbourne et donne de nombreuses indications sur l' identité de tous les jeunes adolescents se produisant aux Transphrase : il s' agit soit à l' arrière d' une limousine décapotable blanche, avec au volant des passagers une sorte " doo grrrrnt hahahohohmmha oui, c' est pour rire (... ahhh! ) De toute facon, les francais disent : tu peux plus pas nous envoyer à la mer sauf nous si on fait des gaffes ou quoi " Et tu les as un peu forcé a le suivre meme qu " 1 semaine aprés y a deja des regains de deboires.. alors au point où vous m ecoutiez jerais pas.. il est venu voir les fg en se branlant le bougre ( lol.. ça le calmeras! ) La ville de Rennes organise chaque printemps sur les pelouses du parc Jean Moulin sa grande " Grande Rencontre de Ville-Égalité 2016 " où les élus locaux, étudiants ou anciens, du personne

- Cette case doit être décommenter si vous effectuez votre entrainement depuis Google colab et que vous avez placé votre dataset depuis la racine du drive.

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [18]:
!ls

gdrive	sample_data


In [5]:
#poemes = pd.read_csv('/content/gdrive/MyDrive/data_new.csv')['texte']
poemes = pd.read_csv('data_new.csv')

---
Faute de ressource, on prend une longueur max de 512, pour pouvoir allouer un espace dans le GPU.\
Lorsqu'on prend une longueur de 768 (le max du modèle), impossible d'allouer cet espace dans le GPU.

---

In [20]:
max_length = max([len(tokenizer.encode(texte)) for texte in poemes])
max_length = min(max_length,768)


In [21]:
class PoemeDataset(Dataset):
    """ """
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            #for english model
            #encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
            #                           max_length=max_length, padding="max_length")
            
            encodings_dict = tokenizer(txt, truncation=True,
                                       max_length=max_length)
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]



In [22]:
dataset = PoemeDataset(poemes, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [23]:
#Garbage collector 
gc.collect()

44

In [24]:
torch.cuda.empty_cache()
torch.cuda.synchronize()
torch.cuda.memory_allocated()

511148032

In [25]:
training_args = TrainingArguments(output_dir='./results',
                                  num_train_epochs=2, 
                                  logging_steps=100, 
                                  save_steps=5000,
                                  per_device_train_batch_size=1, 
                                  per_device_eval_batch_size=1,
                                  warmup_steps=10, 
                                  weight_decay=0.05, 
                                  logging_dir='./logs', 
                                  report_to = 'none')

In [26]:
Trainer(model=model,  args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

***** Running training *****
  Num examples = 32831
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 65662


Step,Training Loss
100,4.435100
200,3.660100
300,3.499400
400,3.590500
500,3.442100
600,3.408100
700,3.441600
800,3.477600
900,3.366500
1000,3.365000


Saving model checkpoint to ./results/checkpoint-5000
Configuration saved in ./results/checkpoint-5000/config.json
Model weights saved in ./results/checkpoint-5000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-10000
Configuration saved in ./results/checkpoint-10000/config.json
Model weights saved in ./results/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-15000
Configuration saved in ./results/checkpoint-15000/config.json
Model weights saved in ./results/checkpoint-15000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-20000
Configuration saved in ./results/checkpoint-20000/config.json
Model weights saved in ./results/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-25000
Configuration saved in ./results/checkpoint-25000/config.json
Model weights saved in ./results/checkpoint-25000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-30000
Configuration saved in ./resu

TrainOutput(global_step=65662, training_loss=2.962423544594381, metrics={'train_runtime': 17013.7283, 'train_samples_per_second': 3.859, 'train_steps_per_second': 3.859, 'total_flos': 8278947194112000.0, 'train_loss': 2.962423544594381, 'epoch': 2.0})

## Génération de quelques exemples avec les modèles entrainés

---
Dans cette partie, nous allons générer quelques exemples pour voir ce que le modèle donne. Nous avons écrit une fonction qui permet de le faire. Pour voir la différence entre le modèle entrainer et non, nous avons effectué une partie bien avant d'entrainer le modèle avec les mêmes phrases au début pour donner un peu le contexte. 

---

In [27]:
generate_text(p1,2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text  0
------------------------------------------
L'amour est un oiseau rebelangé
Une cigale en chemin en solitaire vers midi ;
Il volè sur une aile fragile entre ciel en nuages, et un corbeau qui fait rugir des mots,
Son plumage lui fait écho au crépuscule, tout en s’accroyant avec des claques de main…
Puis les nuages passent, se noient laissant apparaître des rêves aux reflets…
De doux rêves caressant comme des enfants les enfants ; la plume en éveil de ses doux vers me caressent toujours, ma plume sest offerte des rêves des autres…
Dans leurs pensées je rêve. /💫 Un beau ciel qui nous unit enfin : le silence.. »
Le ciel c’est un poème d’adieu… Un tableau sans fin où il existe vraiment, les étoiles… Une belle saison pour les oiseaux du printemps!
Le ciel n est autre qu une goutte à l oeil! Dans ta douce rosée…. Tout près qui me fait toujours espérer, tu me laisses rêver… Une rose… Que tes couleurs rendent folle… De beaux soleils si pétillante! … Comme dans toute relation il est le pr

In [28]:
generate_text(p2,2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text  0
------------------------------------------
Science sans conscience n'est que ruine de l'ame igne sans espoir son règne tu as créer
Tu es parti rejoindre ta mère l homme leur fut fidèle la main lui été a porté
Tu as fuit loin sur nous de ta famille il n aura plu a parler
Et les pleurs coulent alors qu on pleur que c a ton d eux ta faute. Tout a changer ce monde à l arrache comme cela le devient en silence les jour le temp change on c c’qu est aujourd il nous pousse a aller d’otre lieu les larmes à qui ont fait naître nos peur je vous le re demande pardon il ma trop blessé…
Les regrets font tous leurs passages des moi il est pas celui du roi des anges sur terre il c pour son roi que vous levez tous souffri. Sa place dans c empire il c pour son sang je sais ne le trouver ou j apelle… ne pouvant pas mais je sais si je resi qu ici a ce seigneur existe la vie jvous en laisse la trace.. dans l air si bon les gouttes coule et on vous garde et vous vous allez souffrir! vous en ri ou la 

In [29]:
generate_text(p3,2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text  0
------------------------------------------
La haine que l'on ressent }
Lait
Que se propage l onde des orages en se délectant des terres.
Vivre son errance et subir sa folie! et ce
Ceux de ceux qui sont nés, les ancêtres lointains du jour en face!!
Combié par des hommes venus sur terre, par de l’humanité qui en rient!

Nous nous sommes perdus en un cœur sur terre ;
Semant par la fenêtre la poussière qu il avait sur sa faim, en nous volant d espoir comme nous avions été volé avant!
Dans ses vers je voudrai avoir confiance en une grande sagesse car au final si un jour sa vie ne saurait rien, sa soi ce simple nom qui vous est donné ou même tout de rien et plus comme l infini? (en tout çà parce que le temps et ses guerres ont tout emporté pour ce moment la guerre n°as été qu’un grand chaos qu il aura tout dévasté autour à tuer pour cette nouvelle planète dont il n’a plu que fait… pour que je trouve pour de bon, que s’en sortir jai tant mérité avec ce coup dur mais je l"avais laissé

## RNN model

---
Nous allons entrainer un modèle utilisant une architecture RNN, différente de celle basée sur le Transformers. Contrairement au premier modèle, celui-ci est moins lourd.

--- 

### Prepocessing

In [6]:
from collections import Counter
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from argparse import Namespace
import os

In [7]:
poemes

,theme,texte,titre
0,absence,Après la tempête vient le ciel bleu\nAprès la ...,La Perle De Dieu
1,absence,"Au « Logis du p’tit Canette », le 11 octobre 2...",Quand La Lune Se Dessine
2,absence,Le linceul de mes songes dérives lentement\nLa...,Nouveau Souffle
3,absence,"Très jaune de lueurs,\nSur cette faune en sueu...",Ces Étoiles Qui Brillent.
4,absence,"Avant j’avais toujours la tete,\nDans les étoi...",Je Veux Dire Adieux Juste Instant.
...,...,...,...
36474,amitie,C’est une île dans les antilles ;\nArchipel de...,Union
36475,amitie,"Sur mon cœur, tes larmes, tu as déposé\nPour m...",Un Voyage Pour Le Paradis
36476,amitie,"Marchons à deux sur ce chemin,\nSous le soleil...",Un Amour Sans Limites
36477,amitie,"Tu vas voyager\nPour longtemps, t’éloigner\nUn...",Le Voyage


In [8]:
all_text = []
for t in poemes.texte:
    all_text.extend(t.split())

In [9]:
len(set(all_text))

254799

In [10]:
def get_data_from_file(batch_size, seq_size):
    #with open(train_file, 'r', encoding='utf-8') as f:
    #    text = f.read()
    
    text = all_text

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text

In [11]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

### RNN model

In [12]:
class RNNModule(nn.Module):
  
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

In [13]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer

### Training

In [14]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()
    #words = ['Je', 'suis']

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])

    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))
    #print(' '.join(words).encode())

In [15]:
flags = Namespace(
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['Je', 'suis'],
    predict_top_k=5,
    checkpoint_path='checkpoint',
    epoch=1
)

In [16]:
def train():
  
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
        flags.batch_size, flags.seq_size)

    net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)
    net = net.to(device)

    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0

    for e in range(flags.epoch):

        batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
        state_h, state_c = net.zero_state(flags.batch_size)
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            net.train()

            optimizer.zero_grad()

            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            loss_value = loss.item()

            loss.backward()

            state_h = state_h.detach()
            state_c = state_c.detach()

            _ = torch.nn.utils.clip_grad_norm_(
                net.parameters(), flags.gradients_norm)

            optimizer.step()

            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, flags.epoch),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

            if iteration % 1000 == 0:
                predict(device, net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)
                try:
                    torch.save(net.state_dict(),
                           'checkpoint_pt/model-{}.pth'.format(iteration))
                except FileNotFoundError:
                    os.mkdir('checkpoint_pt')

In [17]:
device

'cuda'

In [18]:
train()

Vocabulary size 254799
Epoch: 0/1 Iteration: 100 Loss: 8.55397891998291
Epoch: 0/1 Iteration: 200 Loss: 8.101948738098145
Epoch: 0/1 Iteration: 300 Loss: 7.55696439743042
Epoch: 0/1 Iteration: 400 Loss: 7.960899829864502
Epoch: 0/1 Iteration: 500 Loss: 7.620220184326172
Epoch: 0/1 Iteration: 600 Loss: 7.003274440765381
Epoch: 0/1 Iteration: 700 Loss: 7.468767166137695
Epoch: 0/1 Iteration: 800 Loss: 7.199026107788086
Epoch: 0/1 Iteration: 900 Loss: 7.494446277618408
Epoch: 0/1 Iteration: 1000 Loss: 7.314366340637207
Je suis là, de mes mains se croisent, de mes yeux de la vie est un petit corps est une vie Et les yeux Je me sens en plus un jour qui ne peut pas en peine le cœur Je me sens plus le temps est le ciel est la vie de mes yeux Et le monde se noie Et je suis plus de la tête Et le bonheur Je me laisse à mon corps Je suis plus belle Je n’ai plus le ciel à ce que on me fera à mon corps est si de mes mots et la nuit et des
Epoch: 0/1 Iteration: 1100 Loss: 7.476197719573975
Epoch: 0/

### Génération avec rnn modèle

In [19]:
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file( flags.batch_size, flags.seq_size)

Vocabulary size 254799


In [24]:
net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)

net.load_state_dict(torch.load('checkpoint_pt/model-10000.pth'))
net.to(device)

RNNModule(
  (embedding): Embedding(254799, 64)
  (lstm): LSTM(64, 64, batch_first=True)
  (dense): Linear(in_features=64, out_features=254799, bias=True)
)

In [25]:
predict(device, net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)

Je suis là, de mes mains se croisent, de mes yeux de la vie est un petit corps est une vie Et les yeux Je me sens en plus un jour qui ne peut pas en peine le cœur Je me sens plus le temps est le ciel est la vie de mes yeux Et le monde se noie Et je suis plus de la tête Et le bonheur Je me laisse à mon corps Je suis plus belle Je n’ai plus le ciel à ce que on me fera à mon corps est si de mes mots et la nuit et des fleurs de sa plume Je ne vois jamais un jour je suis un enfant Et que je ne veux plus rien et je n’ai rien de la poésie est un enfant qui ne fait que le ciel qui est la vie Je me suis dit que l’on se trouve de son cœur de mes mots de sa main et moi le temps et de mes mains de nos yeux de nos mots sont en la folie et le ciel de la folie Et de la vie et le monde et la main Je me suis dit si tu es un enfant de la paix Tu me sens si fort Et que je me suis si tu me suis si tu ne sais pas que vous serez le cœur Je ne sais que ça tu ne me laisse rien que j’ai fait des mots et tu me 

In [26]:
predict(device, net, ["Amour", "et", "haine"], n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)

Amour et haine Et je te hais Je te hais le plus beau que j’ai fait des choses et je te suis Je te hais à te dire qu’il est plus tard Tu n’es qu’un peu de toi Je suis toujours là Tu m’as fais pour moi ? Et moi je ne sais que j’ai fait la fin que mon amour a été un jour je suis le plus beau Je te vois pour que j’ai que tu m’as dit de toi Je n’ai rien vu ce qu’est ce monde est le plus que c’est moi Je me laisse dans mes yeux, je ne


In [27]:
predict(device, net, ["Oiseau", "rebel"], n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)

Oiseau rebel et de mon amour et la mort de la lumière Et de mon âme en mon cœur de mon amour Je suis toujours une autre chance Et je n’ai jamais été une seule amie de mon cœur et je me laisse en mon corps Et le jour te dire : Tu m’as dit : - Tu m’as fais la fin des gens Et que j’ai vu mon cœur est devenu le plus que j’ai été que tu m’as donné Mais tu m’as quitté Car tu ne veux pas, Et tu m’as dit que j’ai de moi que tu m’as fais ça
